In [45]:
from selenium import webdriver
import time
import pandas as pd

In [46]:
# betting site URL
web = 'https://sports.tipico.de/de/alle/fussball/deutschland/bundesliga'

# chromedriver
path = 'D:/Download/unzip/chromedriver.exe'

# set a driver instance to navigate through the website
driver = webdriver.Chrome(path)
driver.get(web)

# make selenium to accept cookies on the website
# set implicit wait to stop script for 5 seconds (from time library)
# data is loaded dynamically so it can take some seconds to display all the data correctly.
# As a result, an element might not be located in the DOM (Document Object Model) when scraping the website,
# so we’ll get an “ElementNotVisibleException.
# explicit wait = wait for certain condition to occur
time.sleep(5)

# accept is variable for option button
accept = driver.find_element_by_xpath('//*[@id="_evidon-option-button"]')
accept.click()


time.sleep(2)

# decline is variable for decline button
decline = driver.find_element_by_xpath('//*[@id="evidon-prefdiag-decline"]')
decline.click()


#   --- Building the Web Scraper ---
# empty lists to store all data
teams = []
x12 = []  # 3-way
odds_events = []




# select container with only upcoming matches box
box = driver.find_element_by_xpath(
    '//div[contains(@testid, "Program_SELECTION")]')
time.sleep(5)

# looking for the sports name
sports_title = box.find_elements_by_class_name("SportTitle-styles-sport")

# extract data only from football matches, grandparent determines football container
for sport in sports_title:
    if sport.text == 'Fußball':
        parent = sport.find_element_by_xpath('./..')
        grandparent = parent.find_element_by_xpath('./..')
        # looking for all single row events - produces a list
        single_row_events = grandparent.find_elements_by_class_name(
            'EventRow-styles-event-row')
        # getting odds events and team names
        for match in single_row_events:
            odds_event = match.find_elements_by_class_name(
                'EventOddGroup-styles-odd-groups')
            odds_events.append(odds_event)
            for team in match.find_elements_by_class_name('EventTeams-styles-titles'):
                teams.append(team.text)
         # getting data: the odds
        for o_event in odds_events:
            for n, box in enumerate(o_event):
                rows = box.find_elements_by_xpath('.//*')
                if n == 0:
                    x12.append(rows[0].text)

driver.quit()
# Storing lists within dictionary
dict_gambling = {'Teams': teams, '1x2': x12}

# Presenting data in dataframe
df_gambling = pd.DataFrame.from_dict(dict_gambling)
print(df_gambling)



                                  Teams                1x2
0         FSV Mainz 05\nBor. M'gladbach   2,60\n3,40\n2,70
1           Bayern München\nSC Freiburg  1,20\n7,50\n13,00
2      VfB Stuttgart\nArminia Bielefeld   1,80\n3,80\n4,30
3       VfL Bochum\nTSG 1899 Hoffenheim   3,80\n3,80\n1,90
4            VfL Wolfsburg\nFC Augsburg   1,55\n4,20\n6,00
5         RB Leipzig\nBorussia Dortmund   2,10\n4,00\n3,10
6          Hertha BSC\nBayer Leverkusen   3,80\n3,60\n1,95
7              1. FC Köln\nUnion Berlin   2,20\n3,50\n3,20
8   Greuther Fürth\nEintracht Frankfurt   3,60\n4,00\n1,90
9           FC Augsburg\nBayern München  13,00\n7,50\n1,20
10     Arminia Bielefeld\nVfL Wolfsburg   3,80\n3,60\n1,95
11         Bayer Leverkusen\nVfL Bochum   1,35\n5,00\n9,00
12      Bor. M'gladbach\nGreuther Fürth   1,32\n5,50\n9,00
13     Borussia Dortmund\nVfB Stuttgart   1,40\n5,00\n7,50
14      TSG 1899 Hoffenheim\nRB Leipzig   3,60\n3,80\n1,95
15             Union Berlin\nHertha BSC   2,05\n3,50\n3,